In [ ]:
!pip install datasets --q
!pip install tiktoken --q
!pip install tqdm --q

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tiktoken
import json
import torch
import os
import mmap
import numpy as np
from pathlib import Path
from typing import Optional, Tuple, Dict, Any
import pickle

class OptimizedPrepData:
    """Optimized data preparation class for large text files with caching support."""

    def __init__(self, text_file_path: str, cache_dir: str = "./cache"):
        self.text_file_path = text_file_path
        self.cache_dir = Path(cache_dir)
        self.cache_dir.mkdir(exist_ok=True)

        self.encoder = tiktoken.encoding_for_model('gpt2')

        # Cache file paths
        self.tokens_file = self.cache_dir / "tokens.bin"
        self.vocab_file = self.cache_dir / "vocab.json"
        self.meta_file = self.cache_dir / "meta.json"

        # Initialize or load cached data
        self._initialize_data()

    def _initialize_data(self):
        """Initialize data either from cache or by processing the text file."""
        if self._cache_exists():
            print("Loading from cache...")
            self._load_from_cache()
        else:
            print("Processing text file and creating cache...")
            self._process_and_cache()

    def _cache_exists(self) -> bool:
        """Check if all required cache files exist."""
        return (self.tokens_file.exists() and
                self.vocab_file.exists() and
                self.meta_file.exists())

    def _process_and_cache(self):
        """Process the text file and create cache files."""
        print("Tokenizing text file...")

        # Read and tokenize the file in chunks to handle large files
        chunk_size = 1024 * 1024  # 1MB chunks
        all_tokens = []

        with open(self.text_file_path, 'r', encoding='utf-8') as f:
            while True:
                try:
                    chunk = f.read(chunk_size)
                    if not chunk:
                        break

                    tokens = self.encoder.encode(chunk)
                    all_tokens.extend(tokens)
                    print(f"Processed {len(all_tokens)} tokens so far...", end='\r')
                except Exception as e:
                    print(f"Something went wrong: {e} \n but ignored")
        print(f"\nTotal tokens: {len(all_tokens)}")

        # Create vocabulary mapping
        unique_tokens = sorted(list(set(all_tokens)))
        self.vocab_size = len(unique_tokens)
        self.token_to_idx = {token: idx for idx, token in enumerate(unique_tokens)}
        self.idx_to_token = {idx: token for token, idx in self.token_to_idx.items()}

        print(f"Vocabulary size: {self.vocab_size}")

        # Convert tokens to indices
        print("Converting tokens to indices...")
        token_indices = np.array([self.token_to_idx[token] for token in all_tokens], dtype=np.uint16)

        # Save binary token file
        print("Saving tokens to binary file...")
        with open(self.tokens_file, 'wb') as f:
            f.write(token_indices.tobytes())

        # Save vocabulary mappings
        with open(self.vocab_file, 'w') as f:
            json.dump({
                'token_to_idx': self.token_to_idx,
                'idx_to_token': self.idx_to_token
            }, f)

        # Save metadata
        meta = {
            'total_tokens': len(all_tokens),
            'vocab_size': self.vocab_size,
            'dtype': str(token_indices.dtype)
        }
        with open(self.meta_file, 'w') as f:
            json.dump(meta, f)

        self.total_tokens = len(all_tokens)
        print("Cache created successfully!")

    def _load_from_cache(self):
        """Load data from cache files."""
        # Load metadata
        with open(self.meta_file, 'r') as f:
            meta = json.load(f)

        self.total_tokens = meta['total_tokens']
        self.vocab_size = meta['vocab_size']

        # Load vocabulary
        with open(self.vocab_file, 'r') as f:
            vocab_data = json.load(f)

        # Convert string keys back to integers for idx_to_token
        self.token_to_idx = {int(k): v for k, v in vocab_data['token_to_idx'].items()}
        self.idx_to_token = {int(k): v for k, v in vocab_data['idx_to_token'].items()}

        print(f"Loaded cache: {self.total_tokens} tokens, vocab size: {self.vocab_size}")

class OptimizedTokenizer(OptimizedPrepData):
    """Optimized tokenizer with caching support."""

    def __init__(self, text_file_path: str, cache_dir: str = "./cache"):
        super().__init__(text_file_path, cache_dir)

    def encode(self, text: str) -> list:
        """Encode text to token indices."""
        tokens = self.encoder.encode(text)
        return [self.token_to_idx.get(token, 0) for token in tokens]  # Use 0 for unknown tokens

    def decode(self, token_indices: list) -> str:
        """Decode token indices back to text."""
        tokens = [self.idx_to_token.get(idx, 0) for idx in token_indices]
        return self.encoder.decode(tokens)

class OptimizedDataLoader:
    """Memory-efficient data loader for training."""

    def __init__(self, cache_dir: str, context_length: int, batch_size: int, train_split: float = 0.9):
        self.cache_dir = Path(cache_dir)
        self.context_length = context_length
        self.batch_size = batch_size
        self.train_split = train_split

        # Load metadata
        with open(self.cache_dir / "meta.json", 'r') as f:
            self.meta = json.load(f)

        self.total_tokens = self.meta['total_tokens']
        self.vocab_size = self.meta['vocab_size']

        # Calculate split indices
        self.train_size = int(self.total_tokens * train_split)
        self.val_size = self.total_tokens - self.train_size

        # Memory-map the token file for efficient access
        self.tokens_file = open(self.cache_dir / "tokens.bin", 'rb')
        self.tokens_mmap = mmap.mmap(self.tokens_file.fileno(), 0, access=mmap.ACCESS_READ)

        print(f"DataLoader initialized: {self.total_tokens} tokens, {self.vocab_size} vocab size")
        print(f"Train: {self.train_size}, Val: {self.val_size}")

    def _get_token_at_index(self, idx: int) -> int:
        """Get token at specific index using memory mapping."""
        # Each token is stored as uint16 (2 bytes)
        byte_idx = idx * 2
        return int.from_bytes(self.tokens_mmap[byte_idx:byte_idx+2], byteorder='little')

    def _get_tokens_slice(self, start_idx: int, length: int) -> torch.Tensor:
        """Get a slice of tokens efficiently."""
        tokens = []
        for i in range(start_idx, start_idx + length):
            tokens.append(self._get_token_at_index(i))
        return torch.tensor(tokens, dtype=torch.long)

    def get_batch(self, split: str = 'train', device: str = 'cpu') -> Tuple[torch.Tensor, torch.Tensor]:
        """Get a batch of training data."""
        if split == 'train':
            max_start_idx = self.train_size - self.context_length - 1
            offset = 0
        else:
            max_start_idx = self.val_size - self.context_length - 1
            offset = self.train_size

        # Randomly select starting indices for each sequence in the batch
        start_indices = torch.randint(0, max_start_idx, (self.batch_size,))

        x_batch = []
        y_batch = []

        for start_idx in start_indices:
            actual_idx = offset + start_idx.item()
            x_seq = self._get_tokens_slice(actual_idx, self.context_length)
            y_seq = self._get_tokens_slice(actual_idx + 1, self.context_length)

            x_batch.append(x_seq)
            y_batch.append(y_seq)

        x = torch.stack(x_batch).to(device)
        y = torch.stack(y_batch).to(device)

        return x, y

    def get_sequential_batch(self, split: str = 'train', start_pos: int = 0, device: str = 'cpu') -> Tuple[torch.Tensor, torch.Tensor, int]:
        """Get sequential batches for full dataset processing."""
        if split == 'train':
            max_pos = self.train_size - self.context_length - 1
            offset = 0
        else:
            max_pos = self.val_size - self.context_length - 1
            offset = self.train_size

        if start_pos >= max_pos:
            return None, None, -1  # End of data

        x_batch = []
        y_batch = []

        for i in range(self.batch_size):
            if start_pos + i >= max_pos:
                break

            actual_idx = offset + start_pos + i * self.context_length
            x_seq = self._get_tokens_slice(actual_idx, self.context_length)
            y_seq = self._get_tokens_slice(actual_idx + 1, self.context_length)

            x_batch.append(x_seq)
            y_batch.append(y_seq)

        if not x_batch:
            return None, None, -1

        x = torch.stack(x_batch).to(device)
        y = torch.stack(y_batch).to(device)

        next_pos = start_pos + len(x_batch) * self.context_length

        return x, y, next_pos

    def __del__(self):
        """Clean up memory mapping."""
        if hasattr(self, 'tokens_mmap'):
            self.tokens_mmap.close()
        if hasattr(self, 'tokens_file'):
            self.tokens_file.close()

# Utility functions
def create_optimized_dataset(text_file_path: str, cache_dir: str = "./cache"):
    """Create optimized dataset from text file."""
    prep_data = OptimizedPrepData(text_file_path, cache_dir)
    return prep_data

def load_tokenizer(cache_dir: str = "./cache"):
    """Load tokenizer from cache."""
    # Create a dummy text file path since we're loading from cache
    return OptimizedTokenizer("", cache_dir)

# Example usage
if __name__ == "__main__":
    # Example usage
    text_file_path = "/content/drive/MyDrive/train_data2.txt"
    cache_dir = "./cache"
    context_length = 768
    batch_size = 8

    # Step 1: Create optimized dataset (only runs once, then uses cache)
    prep_data = create_optimized_dataset(text_file_path, cache_dir)

    # Step 2: Create data loader
    data_loader = OptimizedDataLoader(cache_dir, context_length, batch_size)

    # Step 3: Load tokenizer
    tokenizer = load_tokenizer(cache_dir)

    # Step 4: Test the data loader
    print("\nTesting data loader...")
    x, y = data_loader.get_batch('train')
    print(f"Batch shapes: x={x.shape}, y={y.shape}")

    x=x[0].tolist()
    y=y[0].tolist()
    # print(x)
    decoded1 = tokenizer.decode(x)
    decoded2 = tokenizer.decode(y)

    print(f"From batch:\nX:{decoded1[:1000]}\n\nY:{decoded2[:1000]}")
    # Test tokenizer
    print("\nTesting tokenizer...")
    test_text = "the sky is it tasty"
    encoded = tokenizer.encode(test_text)
    decoded = tokenizer.decode(encoded)
    print(f"Original: '{test_text}'")
    print(f"Encoded: {encoded}")
    print(f"Decoded: '{decoded}'")

    print(f"\nDataset info:")
    print(f"Total tokens: {data_loader.total_tokens}")
    print(f"Vocab size: {data_loader.vocab_size}")
    print(f"Train tokens: {data_loader.train_size}")
    print(f"Val tokens: {data_loader.val_size}")

Processing text file and creating cache...
Tokenizing text file...
Something went wrong: 'utf-8' codec can't decode bytes in position 44333-44334: invalid continuation byte 
 but ignored
Something went wrong: 'utf-8' codec can't decode bytes in position 51101-51102: invalid continuation byte 
 but ignored
Something went wrong: 'utf-8' codec can't decode bytes in position 191912-191913: invalid continuation byte 
 but ignored
Something went wrong: 'utf-8' codec can't decode bytes in position 7495-7496: invalid continuation byte 
 but ignored
Something went wrong: 'utf-8' codec can't decode bytes in position 755-756: invalid continuation byte 
 but ignored
Something went wrong: 'utf-8' codec can't decode bytes in position 60801-60802: invalid continuation byte 
 but ignored
Something went wrong: 'utf-8' codec can't decode bytes in position 241325-241326: invalid continuation byte 
 but ignored
Something went wrong: 'utf-8' codec can't decode bytes in position 434-435: invalid continuatio

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
# from model import TLMBlock
# from g_mlp_pytorch import gMLP
# from Data import load_tokenizer
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
# torch.set_default_device('cpu')

class TLMBlock(nn.Module):

  def __init__(self, num_heads, context_length, embed_size):
    super().__init__()

    self.ln_1 = nn.LayerNorm(embed_size)
    self.sa_head = MultiHeadedAttention(num_heads, context_length,embed_size)
    self.dropout = nn.Dropout(p=0.2)
    self.ln_2 = nn.LayerNorm(embed_size)
    # self.silu = nn.SiLU()
    self.mlp = nn.Sequential(
      nn.Linear(embed_size, 2*embed_size),
      nn.GELU(),
      nn.Dropout(p=0.1),
      nn.Linear(2*embed_size, embed_size),
    )
  def forward(self, x):

    # B,T = x.shape
    # print(B,T)
    x = x+self.sa_head(self.ln_1(x))
    # print(x.shape)
    x = x + self.mlp(self.ln_2(x))

    # print(x.shape)
    return x

class SpecialTimedDecoderBlock(nn.Module):

    def __init__(self, timesteps, num_heads, context_length, embed_size, device='cpu'):
       super().__init__()

       self.device = device

       self.embeddings = nn.Embedding(num_embeddings=timesteps, embedding_dim=embed_size)
       self.LLMBlock = TLMBlock(num_heads, context_length, embed_size)
    #    self.timer = torch.tensor([0]).to(device=device)
    #    self.timer.requires_grad_(False)

       self.time_steps = timesteps

    def forward(self, current_embs):
        timer = torch.tensor([i for i in range(self.time_steps)]).to(self.device)
        new_embs = current_embs
        for i in range(self.time_steps):
            # timer[0] = i
            # print(timer[i].unsqueeze(0))
            current_embs = current_embs + new_embs + self.embeddings(timer[i].unsqueeze(0))
            new_embs = self.LLMBlock(current_embs)

        return new_embs

class Head(nn.Module):

    def __init__(self, context_length, embed_size, head_dim):
        super().__init__()

        self.queries = nn.Linear(embed_size, head_dim, bias = False)
        self.keys = nn.Linear(embed_size, head_dim, bias=False)
        self.values = nn.Linear(embed_size, head_dim, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(context_length, context_length)))

        # self.ln = nn.LayerNorm(head_dim)
        # self.ln2 = nn.LayerNorm(head_dim)
        self.head_dim = head_dim
    def forward(self, X):

        B,T,C = X.shape
        q = self.queries(X)
        k = self.keys(X)
        # wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = q @ k.transpose(-2, -1) * self.head_dim**-0.5
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf'))
        wei = F.softmax(wei, dim = -1)
        v = self.values(X)

        out = wei@v
        # print(out.shape)
        return out


class MultiHeadedAttention(nn.Module):

    def __init__(self, num_heads, context_length, embed_size):
        super().__init__()

        self.heads = nn.ModuleList([Head(context_length, embed_size, embed_size//num_heads) for _ in range(num_heads)])
        self.fc = nn.Linear(embed_size,embed_size)
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim =-1)
        out = self.fc(out)
        return out

class RecurrentLM(nn.Module):

  def __init__(self, vocab_size, context_length, embed_size, num_blocks = 5, device = 'cpu'):
    super().__init__()
    self.token_embeddings = nn.Embedding(vocab_size, embed_size)
    self.positional_embeddings = nn.Embedding(context_length, embed_size)
    self.word_embeddings = nn.Embedding(context_length, embed_size)
    self.context_length = context_length

    self.blocks = SpecialTimedDecoderBlock(num_blocks, 4, context_length, embed_size, device=device)

    # self.block = nn.ModuleList()
    # for _ in range(num_blocks):
    #   self.block.append(TLMBlock( 4, context_length,embed_size))
    self.device = device
    self.lm_head = nn.Linear(embed_size,vocab_size)
    self.silu = nn.SiLU()

  def forward(self, idx, targets=None):
    B,T = idx.shape
    # print(idx.shape)
    # print(B,T)
    tok_emb = self.token_embeddings(idx)
    # print(tok_emb.shape)
    pos_emb = self.positional_embeddings(torch.arange(T, device=self.device))
    # word_emb = self.word_embeddings(idx)
    x = tok_emb + pos_emb
    # print(x.shape)
    # for _,layer in enumerate(self.block):
    #   x = layer(x)
    x = self.blocks(x)

    logits = self.lm_head(x)
    if targets == None:
      loss = None
    else:
      B,T,C = logits.shape
      logits = logits.view(B*T, C)
      # print(logits.shape)
      targets = targets.view(B*T)
      # print(targets.shape)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
      for _ in range(max_new_tokens):

        idx_cond = idx[:,-self.context_length:]

        logits, loss = self(idx_cond)

        logits = logits[:,-1,:]

        probs = F.softmax(logits, dim=-1)

        idx_next = torch.multinomial(probs, num_samples=1)

        idx = torch.cat((idx, idx_next), dim = 1)
        # print(idx.shape)
      return idx

def generate(string):

  for s in string:
    print(s, end="")
    time.sleep(0.05)

if __name__ == "__main__":

    # prev_embs = torch.randn((1, 768, 1024)).to(device="cuda")

    # specialblock = SpecialTimedDecoderBlock(5, 4, 768, 1024, device='cuda')
    # specialblock = specialblock.to(device='cuda')
    # out = specialblock(prev_embs)
    # print(out)
    # print(out.shape)

    tlm = RecurrentLM(4,8,32,10,device=device)
    tlm = tlm.to(device)
    x = torch.ones((8,4),dtype=torch.long).to(device)
    out = tlm(x)
    print(out[0].shape)

torch.Size([8, 4, 4])


In [ ]:
# example_optimized.py
# from Data import OptimizedDataLoader, create_optimized_dataset, load_tokenizer
# from model import TLM        # Your existing model
import torch
from tqdm import tqdm

def main():
    # Configuration
    text_file_path = "/content/drive/MyDrive/train_data2.txt"
    cache_dir = "./cache"
    context_length = 768
    batch_size = 16
    n_embs = 1024

    # Device setup
    dev = 'cuda' if torch.cuda.is_available() else 'cpu'
    device = torch.device(dev)
    print(f"Using device: {device}")

    # Step 1: Create optimized dataset (runs once, then uses cache)
    print("Setting up dataset...")
    prep_data = create_optimized_dataset(text_file_path, cache_dir)

    # Step 2: Create data loader
    print("Creating data loader...")
    data_loader = OptimizedDataLoader(cache_dir, context_length, batch_size)

    # Step 3: Load tokenizer
    tokenizer = load_tokenizer(cache_dir)

    # Step 4: Initialize model (using your existing model)
    print("Initializing model...")
    m = RecurrentLM(data_loader.vocab_size, context_length, n_embs, num_blocks=12, device=device)
    m.to(device)

    # Step 5: Initialize trainer (using your existing trainer)
    # trainer = Trainer(100, 10, device=dev)

    optimizer = torch.optim.AdamW(m.parameters(), lr=0.001)

    # Step 6: Custom training loop with optimized data loading
    print("\nStarting training...")

    # Example training loop
    num_epochs = 100
    steps_per_epoch = 50
    # save_step = 10
    c = 900

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch + 1}/{num_epochs}")
        total_loss = 0
        # Training
        for step in tqdm(range(steps_per_epoch), "Steps"):
            # Get batch efficiently
            x, y = data_loader.get_batch('train', device=dev)

            # Your training step here
            xb = x.to(device)
            yb = y.to(device)
            logits, loss = m(xb,yb)
            total_loss += loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            torch.cuda.empty_cache()
            # if steps_per_epoch%save_step == 0 and steps_per_epoch != 0:
            #     print(f"step:{steps_per_epoch} loss:{loss}")

            # if step % 9 == 0:
            #     print(f"  Step {step}/{steps_per_epoch}, Batch shape: {x.shape}")


        if c > loss.item():
            c = loss.item()
            torch.save(m.state_dict(), "/content/drive/MyDrive/model2.pt")

        print(f"total_loss_per_epoch: {total_loss/steps_per_epoch}")

        # Validation
        print("  Running validation...")
        val_steps = 1
        for step in range(val_steps):
            x_val, y_val = data_loader.get_batch('val', device=dev)
            with torch.no_grad():
                initial_text = "Once upon a time"
                context = torch.tensor([tokenizer.encode(initial_text)], dtype=torch.long).to(device)
                m.generate(context, 100)
                generated_tokens = m.generate(context, 100)[0].tolist()
                generated_text = tokenizer.decode(generated_tokens)
                print(f"Generated: {generated_text[:1000]}")
            # val_loss = trainer.eval_step(m, x_val, y_val)

    # Step 7: Generate text
    print("\nGenerating text...")

    # Create initial context
    initial_text = "Once upon a time"
    context = torch.tensor([tokenizer.encode(initial_text)], dtype=torch.long).to(device)

    # Generate (using your existing model)
    generated_tokens = m.generate(context, 100)[0].tolist()
    generated_text = tokenizer.decode(generated_tokens)
    print(f"Generated: {generated_text}")

    print("\nDataset Statistics:")
    print(f"Total tokens: {data_loader.total_tokens:,}")
    print(f"Vocabulary size: {data_loader.vocab_size:,}")
    print(f"Train tokens: {data_loader.train_size:,}")
    print(f"Validation tokens: {data_loader.val_size:,}")
    print(f"Context length: {context_length}")
    print(f"Batch size: {batch_size}")

if __name__ == "__main__":
    main()

Using device: cuda
Setting up dataset...
Loading from cache...
Loaded cache: 9108093 tokens, vocab size: 11799
Creating data loader...
DataLoader initialized: 9108093 tokens, 11799 vocab size
Train: 8197283, Val: 910810
Loading from cache...
Loaded cache: 9108093 tokens, vocab size: 11799
Initializing model...

Starting training...

Epoch 1/100


Steps: 100%|██████████| 50/50 [02:22<00:00,  2.86s/it]


total_loss_per_epoch: 5331.8837890625
  Running validation...
Generated: Once upon a time, Timmy's try to help him?" Lily was scared and Tim was just some cake hard lesson. One day, she said.
Suddenly, there was so happy and he didn't win." Her fish and Spot and he heard Lily was so she loved to chase
Ben felt on his tail. Timmy was a big was a bug that seem," Mom asks. His mom to cut is your next day ever had stood come true learned that's parent books that and play something sang clean up

Epoch 2/100


Steps: 100%|██████████| 50/50 [02:30<00:00,  3.01s/it]


total_loss_per_epoch: 2487.562255859375
  Running validation...
Generated: Once upon a time, Spot was a little girl named Lily put her mommy saw that she was grateful for looking at it. It made her mommy said, there was a time, they started to Max was big and she felt sad. One day. One day, they would make her mommy saw her mommy loved to life. One day. She said, a time, they couldn't remember was a time, a serves. From then her fault horse. Sour. It made him nicely, they loved

Epoch 3/100


...

Steps: 100%|██████████| 50/50 [02:31<00:00,  3.04s/it]


total_loss_per_epoch: 951.5985717773438
  Running validation...
Generated: Once upon a time, there was time, a little girl had escaped from her story time was happy. One day, mommy's mommy saw a time, but Nemo found, but Timmy and wanted to keep her on Tuna around and quacked and Ann. From that her house. She had a big, Timmy felt proud back to play with me?" asked her mom bought her to clean it would always keep her colorful crayons. She goes first, but her mommy had to play

Epoch 35/100


Steps: 100%|██████████| 50/50 [02:31<00:00,  3.03s/it]


total_loss_per_epoch: 891.229736328125
  Running validation...
Generated: Once upon a time, how safe and Mia was time, Benny if you, Sara there was broken pieces say, mommy felt safe and teaching about her time, wondering that day, Sara," Leo flew playing with his owner sealed and keep his favorite one day, his room singing there was walking, Lily and games! Splash and much."
After playing doctor said, there was so loudly. He played with hissed loudly and rude and forgot all the bathtub. He would fly. You are sincere and did not

Epoch 36/100


Steps:  66%|██████▌   | 33/50 [01:40<00:51,  3.03s/it]